# Master Cell 

Everything that works is addded to the cell below, you can complete 100% of the scraping by running it 

In [22]:
#started at 1:19 pm, 4/10
#started at 3:06 pm, 4/26

import requests
from bs4 import BeautifulSoup
import pandas as pd
import selenium
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time

#start by getting the strain names/links
URL = 'https://www.leafly.com/strains?sort=name'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

href = []
for x in range(1, 115):
    #loop through all strain webpages
    URL = 'https://www.leafly.com/strains?sort=name&page='+ str(x)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #get arrays containing strain info
    results = soup.find(id='__next')
    #get href out of arrays
    for i in results.find_all('a'):
        href.append(i.get('href'))

#remove everything that isn't a strain
drop = ['/news/cannabis-101/sativa-indica-and-hybrid-differences-between-cannabis-types',
     '/strains/lists/category/sativa',
     '/strains/lists/category/indica',
     '/strains/lists/category/hybrid',
     'https://www.leafly.com/',
     'https://www.leafly.com/strains']

for x in range(1, 115):
    drop.append('/strains?sort=name&page=' + str(x))
for y in drop:
    href.remove(y)
href = [e for e in href if e not in drop]

def clean_names(x):
    '''it is easier to clean the names from the href than it is to scrape them'''
    if 'strains' in x:
        x = x.replace('strains', ',')
    x = x.replace('/', '')
    x = x.replace('-', ' ')
    x = x.replace(',', ', ')
    x = x.title()
    if x[0] ==',':
        x = x[1:]
    if x[0] ==' ':
        x = x[1:]
    return x

#go to each strain page and get the goods
#save all our information
info = []

for g in href:
    
    #go to the page for the strain
    URL = 'https://www.leafly.com'+g
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')

    #get description
    description = []
    des = soup.find('div', attrs={'class': "md:mb-xxl strain__description relative overflow-hidden"})
    if des is not None:
        description = des.p
        if description is not None:
            description = description.text

    #get name
    name = clean_names(g)

    #get terpene info 
    bar = soup.find('div', attrs={'class': "flex flex-row flex-grow mt-xs"})

    tech_terp = []
    terpene_description = []
    terpene_des = []
    #some strains have no terpene info, we need to skip through those
    if bar is not None:
        for i in bar:
            #this one was easy, just getting the text out of the div
            terpene_description= i.find('div', attrs={'class': 'font-normal'})
            terpene_des.append(terpene_description.text)

            #this is to get the info out of the style bar, those two contain the real life name and percentage of bar they represent
            tech_terp.append(i.attrs['style'])
            tech_terp.append(i.attrs['data-testid'])
    else:
        pass

    
    #okay so now to get the lineage 
    line = soup.find('div', attrs={'class': 'font-bold font-headers lineage__container px-lg text-sm'})
    
    #not every strain has lineage 
    
    #there are no parents, two, or one
    parents = []
    left_parent = []
    right_parent = []
    single_parent = []
    children = []
    left_child = []
    right_child = []
    only_child = []
    left_single_p_child = []
    right_single_p_child = []

    if line is not None:

        left_parent = line.find('div', attrs={'class': 'lineage__left-parent'})
        if left_parent is not None:
            get_array = left_parent.find('a')
            parents = get_array.get('href')

        right_parent = line.find('div', attrs={'class': 'lineage__right-parent'})
        if right_parent is not None:
            get_array = right_parent.find('a')
            parents += (get_array.get('href'))

        single_parent = line.find('div', attrs={'class': 'lineage__center-parent'})
        if single_parent is not None:
            get_array = single_parent.find('a')
            parents = (get_array.get('href'))

        if parents != []:
            parents = clean_names(parents)

        #most of the strains have two children, but some only have one 

        left_child = line.find('div', attrs={'class': 'lineage__left-child--two-parents'})
        if left_child is not None:
            get_array = left_child.find('a')
            children = (get_array.get('href'))

        right_child = line.find('div', attrs={'class': 'lineage__right-child--two-parents'})
        if right_child is not None:
            get_array = right_child.find('a')
            children += (get_array.get('href'))

        #for the only children, single parent, no grandparents households out there
        only_child = line.find('div', attrs={'class': 'lineage__center-child--no-parents'})
        if only_child is not None:
            get_array = only_child.find('a')
            children = (get_array.get('href'))

        #children of parents with one parent 
        left_single_p_child = line.find('div', attrs={'class': 'lineage__left-child--single-parent'})
        if left_single_p_child is not None:
            get_array = left_single_p_child.find('a')
            children = (get_array.get('href'))

        right_single_p_child = line.find('div', attrs={'class': 'lineage__right-child--single-parent'})
        if right_single_p_child is not None:
            get_array = right_single_p_child.find('a')
            children += (get_array.get('href'))

        if children != []:
            children = clean_names(children)
            
    
    #get images
    pic = []
    img = soup.find('meta', attrs={'itemprop': 'image'})
    if img is not None:
        pic = img.attrs['content']
        
    #getting thc content 
    thc_content = []
    thc = soup.find('div', attrs={'class':'font-body'})
    if thc is not None:
        thc_content = thc.text
        
    #now get rating
    rating = []
    rate = soup.find('div', attrs={'class':'self-end'})
    if rate is not None:
        rate = rate.p
        if rate is not None:
            rating = rate.text
    
    #get reported feelings via the reviews
    #connect to chrome
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)

    #click the load button until it doesn't exist anymore
    #also gets through age gate
    #and closes newsletter pop up
    #this took so damn long 
    url = "https://www.leafly.com"+g+"/reviews"
    driver.get(url)
    try:
        driver.find_element_by_xpath("//button[@data-testid='age-gate-yes-button']").click()
    except:
        pass
    try:
        if driver is not None:
            more_buttons = driver.find_element_by_xpath("//button[text()='load more']")

            while True:
                try:
                    if more_buttons.is_displayed():
                        driver.execute_script("arguments[0].click();", more_buttons)
                        time.sleep(1)
                except StaleElementReferenceException:
                    break
                try:
                    driver.find_element_by_xpath("/html/body/div[1]/div[8]/div/button").click()
                except:
                    pass
                try:
                    if more_buttons.is_displayed():
                        driver.execute_script("arguments[0].click();", more_buttons)
                        time.sleep(1)
                except StaleElementReferenceException:
                    break
                
    except NoSuchElementException:
        pass


    page_source = driver.page_source
    
    if page_source is not None:

        #get the reported feelings for each strain
        soup = BeautifulSoup(page_source, 'lxml')
        w = soup.find_all('span', attrs={'class' : 'tag mr-sm'})

        reported_feelings = []
        for i in w:
            reported_feelings.append(i.text)

        #finally, save it all in a list
        info.append([name, description, terpene_des, tech_terp, parents, children, reported_feelings, pic, thc_content, rating])

        #so I can save it to a dataframe 
        df = pd.DataFrame(info, columns=['name', 'description', 'terpene_description', 'technical_terpenes', 'parents',
                                        'children', 'reported_feelings', 'pic', 'thc_content', 'rating'])
        df.to_csv('leafly_info_cont_peanut_.csv')
    else:
        pass


     #   print('Strain name:', name)
     #   print('Description:', description)
     #   print('Terpene Description:', terpene_des)
     #   print('Terpene Details:', tech_terp)
     #   print('Parents:', parents)
     #   print('Children:', children)
     #   print('Reported feelings:', reported_feelings)

    df.head()

In [ ]:
#I am going just scraping partial info 
#just reported feelings, images, thc content, and rating

import requests
from bs4 import BeautifulSoup
import pandas as pd
import selenium
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException
 
"""
#start by getting the strain names/links
URL = 'https://www.leafly.com/strains?sort=name'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

href = []
for x in range(1, 115):
    #loop through all strain webpages
    URL = 'https://www.leafly.com/strains?sort=name&page='+ str(x)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #get arrays containing strain info
    results = soup.find(id='__next')
    #get href out of arrays
    for i in results.find_all('a'):
        href.append(i.get('href'))

#remove everything that isn't a strain
drop = ['/news/cannabis-101/sativa-indica-and-hybrid-differences-between-cannabis-types',
     '/strains/lists/category/sativa',
     '/strains/lists/category/indica',
     '/strains/lists/category/hybrid',
     'https://www.leafly.com/',
     'https://www.leafly.com/strains']

for x in range(1, 115):
    drop.append('/strains?sort=name&page=' + str(x))
for y in drop:
    href.remove(y)
href = [e for e in href if e not in drop]

def clean_names(x):
    '''it is easier to clean the names from the href than it is to scrape them'''
    if 'strains' in x:
        x = x.replace('strains', ',')
    x = x.replace('/', '')
    x = x.replace('-', ' ')
    x = x.replace(',', ', ')
    x = x.title()
    if x[0] ==',':
        x = x[1:]
    if x[0] ==' ':
        x = x[1:]
    return x
"""
info = []

for g in href:
    
    #get name
    name = clean_names(g)
    
    #get reported feelings via the reviews
    #connect to chrome
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    #options.add_argument('--incognito')
    #options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)

    #click the load button until it doesn't exist anymore
    url = "https://www.leafly.com"+g+"/reviews"
    url = "https://www.leafly.com"+g+"/reviews"
    driver.get(url)
    driver.find_element_by_xpath("//button[@data-testid='age-gate-yes-button']").click()
    try:
        if driver is not None:
            more_buttons = driver.find_element_by_xpath("//button[text()='load more']")

            while True:
                try:
                    if more_buttons.is_displayed():
                        driver.execute_script("arguments[0].click();", more_buttons)
                        time.sleep(1)
                except StaleElementReferenceException:
                    break
                try:
                    driver.find_element_by_xpath("/html/body/div[1]/div[8]/div/button").click()
                except:
                    pass
                try:
                    if more_buttons.is_displayed():
                        driver.execute_script("arguments[0].click();", more_buttons)
                        time.sleep(1)
                except StaleElementReferenceException:
                    break
                
    except NoSuchElementException:
        pass


    page_source = driver.page_source

    #get the reported feelings for each strain
    soup = BeautifulSoup(page_source, 'lxml')
    w = soup.find_all('span', attrs={'class' : 'tag mr-sm'})

    reported_feelings = []
    for i in w:
        reported_feelings.append(i.text)

    #get images
    pic = []
    img = soup.find('meta', attrs={'itemprop': 'image'})
    if img is not None:
        pic = img.attrs['content']
        
    #getting thc content 
    thc_content = []
    thc = soup.find('div', attrs={'class':'font-body'})
    if thc is not None:
        thc_content = thc.text
        
    #now get rating
    rating = []
    rate = soup.find('div', attrs={'class':'self-end'})
    if rate is not None:
        rate = rate.p
        if rate is not None:
            rating = rate.text

    #finally, save it all in a list
    info.append([name, reported_feelings, pic, thc_content, rating])
    
    #so I can save it to a dataframe 
    df = pd.DataFrame(info, columns=['name', 'reported_feelings', 'pic', 'thc_content', 'rating'])
    df.to_csv('leafly_feelings.csv')

In [ ]:
#this is the pop up! finally found it 

<button class="absolute mr-lg mt-lg newsletter__close right-0 top-0" aria-label="Close newsletter signup modal">

### getting images, THC content, and rating

In [123]:
#first get images

img = soup.find('meta', attrs={'itemprop': 'image'})
if img is not None:
    pic = img.attrs['content']
    
pic

#remember to clean out pngs when cleaning/merging this data

'https://public.leafly.com/strains/flowers/2-fast-2-vast-flower.png'

In [122]:
#getting thc content 

thc = soup.find('div', attrs={'class':'font-body'})
if thc is not None:
    thc_content = thc.text
    
thc_content

#add to cleaning, removing following text when no thc content
#'This info is sourced from our readers and is not a substitute for professional medical advice. 
#Seek the advice of a health professional before using cannabis for a medical condition.'

'This info is sourced from our readers and is not a substitute for professional medical advice. Seek the advice of a health professional before using cannabis for a medical condition.'

In [121]:
#now get rating

rate = soup.find('div', attrs={'class':'self-end'})
if rate is not None:
    rate = rate.p
    if rate is not None:
        rating = rate.text
rating
        
#add to cleaning:
#remove 'Write a review' when no rating 
#take the first 3 characters of the string(contains actual rating)

'Write a review'

### Troubleshooting why I got the wrong images

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

URL = 'https://www.leafly.com/strains/blue-dream'
page = requests.get(URL)
time.sleep(5)
soup = BeautifulSoup(page.content, 'html.parser')

In [10]:
img_cont = soup.findAll('li', attrs={'class':'slider-slide'})
print(img_cont[0])
print()
print(img_cont[1])
print()
print(img_cont[2])

<li class="slider-slide slide-visible" style="box-sizing:border-box;display:inline-block;height:auto;left:0;list-style-type:none;margin-bottom:auto;margin-left:0;margin-right:0;margin-top:auto;-moz-box-sizing:border-box;position:absolute;top:0;transform:scale(1);transition:transform .4s linear;vertical-align:top;width:0" tabindex="-1"><img alt="Picture of Blue Dream strain" class="lazyload m-auto bg-white" data-src="https://leafly-public.imgix.net/web-strain-explorer/1x1-ffffffff.png" height="440" src="https://leafly-public.imgix.net/web-strain-explorer/1x1-ffffffff.png" width="440"/></li>

<li class="slider-slide" style="box-sizing:border-box;display:inline-block;height:auto;left:0;list-style-type:none;margin-bottom:auto;margin-left:0;margin-right:0;margin-top:auto;-moz-box-sizing:border-box;position:absolute;top:0;transform:scale(1);transition:transform .4s linear;vertical-align:top;width:0" tabindex="-1"><img alt="Leafly flower of Blue Dream" class="m-auto bg-white" src="https://pub

In [8]:
soup.findAll('img', attrs={'class': 'm-auto bg-white ls-is-cached lazyloaded'})
#this is what I actually need from the website to get the exact image I want
#but I need to find a way around their lazyloading

[]

In [ ]:
#what I scrape

<img alt="Picture of Blue Dream strain" class="lazyload m-auto bg-white" 
data-src="https://leafly-public.imgix.net/web-strain-explorer/1x1-ffffffff.png" 
height="440" src="https://leafly-public.imgix.net/web-strain-explorer/1x1-ffffffff.png" width="440"/></li>

In [ ]:
#what is actually on the website

<img src="https://leafly-production.imgix.net/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fleafly-images%2Fflower-images%2Fblue-dream.png?w=440&amp;h=440&amp;auto=format&amp;fit=crop&amp;dpr=2&amp;q=25&amp;ixlib=js-2.3.1&amp;s=4fd778910e03234e4cee02cda5780f2b"
data-src="https://leafly-production.imgix.net/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fleafly-images%2Fflower-images%2Fblue-dream.png?w=440&amp;h=440&amp;auto=format&amp;fit=crop&amp;dpr=2&amp;q=25&amp;ixlib=js-2.3.1&amp;s=4fd778910e03234e4cee02cda5780f2b" 
alt="Picture of Blue Dream strain" width="440" height="440" class="m-auto bg-white ls-is-cached lazyloaded ">

In [ ]:
#from a page I did not click on at all, hoping to see what is there without triggering
#js functions with scroll/click/etc

<meta itemprop="image" content="https://s3-us-west-2.amazonaws.com/leafly-images/flower-images/blue-dream.png">

In [13]:
take_2 = soup.findAll('meta', attrs={'itemprop':"image"})
print(take_2)

#awesome, this is bringing me a real image
#time to test with more strains

[<meta content="https://s3-us-west-2.amazonaws.com/leafly-images/flower-images/blue-dream.png" itemprop="image"/>]


In [16]:
#testing new img method with just 5 out of 115 web pages worth of strains
#it went well, not many strains have pictures but the ones that do are showing up!!
#starting with complete strain list at 6:32pm, 6/4/20

URL = 'https://www.leafly.com/strains?sort=name'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

href = []
for x in range(1, 115):
    #loop through all strain webpages
    URL = 'https://www.leafly.com/strains?sort=name&page='+ str(x)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #get arrays containing strain info
    results = soup.find(id='__next')
    #get href out of arrays
    for i in results.find_all('a'):
        href.append(i.get('href'))

#remove everything that isn't a strain
drop = ['/news/cannabis-101/sativa-indica-and-hybrid-differences-between-cannabis-types',
     '/strains/lists/category/sativa',
     '/strains/lists/category/indica',
     '/strains/lists/category/hybrid',
     'https://www.leafly.com/',
     'https://www.leafly.com/strains']

for x in range(1, 115):
    drop.append('/strains?sort=name&page=' + str(x))
for y in drop:
    href.remove(y)
href = [e for e in href if e not in drop]

def clean_names(x):
    '''it is easier to clean the names from the href than it is to scrape them'''
    if 'strains' in x:
        x = x.replace('strains', ',')
    x = x.replace('/', '')
    x = x.replace('-', ' ')
    x = x.replace(',', ', ')
    x = x.title()
    if x[0] ==',':
        x = x[1:]
    if x[0] ==' ':
        x = x[1:]
    return x

info = []

for g in href:
    
    #go to the page for the strain
    URL = 'https://www.leafly.com'+g
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #get name
    name = clean_names(g)
    
    picture = soup.findAll('meta', attrs={'itemprop':"image"})
    
    #finally, save it all in a list
    info.append([name, picture])

    #so I can save it to a dataframe 
    df = pd.DataFrame(info, columns=['name', 'picture'])
    df.to_csv('just_pictures.csv')

### Getting around the load more button for reviews 

In [11]:
import requests

URL = "https://www.leafly.com/strains/100-og/reviews"
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

button = soup.findAll('button', attrs={'class': "button"})
button

for i in button:
    if i.text == 'load more':
        the_button = i

In [12]:
button

[<button class="button" data-testid="age-gate-no-button" id="tou-cancel" tabindex="0">no</button>,
 <button class="button ml-md" data-testid="age-gate-yes-button" id="tou-continue" tabindex="0">yes</button>,
 <button class="button mx-auto">sign up</button>,
 <button class="button">load more</button>]

In [41]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException

url = "https://www.leafly.com/strains/100-og/reviews"

driver = webdriver.Chrome()
driver.get(url)

#try:
button = driver.find_element_by_xpath("//button[@data-testid='age-gate-yes-button']")
#cross = driver.find_element_by_id("nvpush_cross")
print(button)
#button.click()
#except:
   # pass

#while True:
#    try:
#        loadmore = driver.find_element_by_xpath("//button/[text()='load more']/parent::div[@class='w-full flex items-center justify-center lg:justify-start']")
#        loadmore.click()
#    except NoSuchElementException:
#        print("Reached bottom of page")
#        break

<selenium.webdriver.remote.webelement.WebElement (session="a084c09a6f3f9b3e52ddd513a1ce3fb2", element="a9c5a015-0f00-44ea-92fc-1cec95be9be4")>


In [ ]:
import selenium
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
import time

#connect to chrome
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
#options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

#click the load button until it doesn't exist anymore
driver.get("https://www.leafly.com/strains/100-og/reviews")
more_buttons = driver.find_element_by_xpath("//button[text()='load more']")

while True:
    try:
        if more_buttons.is_displayed():
            driver.execute_script("arguments[0].click();", more_buttons)
            time.sleep(1)
    except StaleElementReferenceException:
        print('maybe no more pages')
        break
    
page_source = driver.page_source

#get the reported feelings for each strain
soup = BeautifulSoup(page_source, 'lxml')
w = soup.find_all('span', attrs={'class' : 'tag mr-sm'})

reported_feelings = []
for i in w:
    reported_feelings.append(i.text)
    
reported_feelings